In [3]:
import pandas as pd
import numpy as np

# 1. LOAD (Already uploaded)
df = pd.read_csv('/content/games.csv', on_bad_lines='skip', index_col=False)

# 2. FILTER (2025-2026)
df['release_date_clean'] = pd.to_datetime(df['Release date'], errors='coerce')
df_recent = df[df['release_date_clean'] >= '2025-01-01'].copy()

# 3. MATH (Midpoint Owners & Leakage)
def fix_owners(value):
    if isinstance(value, str):
        parts = value.replace(',', '').split(' - ')
        if len(parts) == 2:
            return (int(parts[0]) + int(parts[1])) / 2
    return 0

df_recent['owners_midpoint'] = df_recent['Estimated owners'].apply(fix_owners)
df_recent['total_reviews'] = df_recent['Positive'] + df_recent['Negative']
df_recent = df_recent[df_recent['total_reviews'] > 0]

df_recent['leakage_rate'] = df_recent['Negative'] / df_recent['total_reviews']
df_recent['revenue_estimated'] = df_recent['owners_midpoint'] * df_recent['Price']
df_recent['revenue_at_risk'] = df_recent['leakage_rate'] * df_recent['revenue_estimated']

# 4. THE EXPLOSION (Splitting the Tags)
# This turns "Action, RPG" into two separate rows so Excel can count them individually
df_recent['Tags'] = df_recent['Tags'].astype(str).str.split(',')
df_exploded = df_recent.explode('Tags')
df_exploded['Tags'] = df_exploded['Tags'].str.strip()

# 5. SAVE
df_exploded.to_csv('steam_tags_exploded.csv', index=False)
print("Done! Download 'steam_tags_exploded.csv' from the sidebar.")

Done! Download 'steam_tags_exploded.csv' from the sidebar.
